In [20]:
# To do: 

# Drop Cloumns
# Date Modification
# Price Correction
# String conversion

# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import datetime as dt
import numpy as np

#  Import and read the file:
import pandas as pd 
super_df = pd.read_csv("resources/Global Superstore Orders 2016.csv")
super_df.head()

Row ID                  Order ID Order Date Ship Date     Ship Mode  \
0   40098  CA-2014-AB10015140-41954   11/11/14  11/13/14   First Class   
1   26341    IN-2014-JR162107-41675     2/5/14    2/7/14  Second Class   
2   25330    IN-2014-CR127307-41929   10/17/14  10/18/14   First Class   
3   13524   ES-2014-KM1637548-41667    1/28/14   1/30/14   First Class   
4   47221   SG-2014-RH9495111-41948    11/5/14   11/6/14      Same Day   

    Customer ID     Customer Name      Segment  Postal Code           City  \
0  AB-100151402     Aaron Bergman     Consumer      73120.0  Oklahoma City   
1     JR-162107     Justin Ritter    Corporate          NaN     Wollongong   
2     CR-127307      Craig Reiter     Consumer          NaN       Brisbane   
3    KM-1637548  Katherine Murray  Home Office          NaN         Berlin   
4    RH-9495111       Rick Hansen     Consumer          NaN          Dakar   

   ...   Product ID    Category Sub-Category  \
0  ...  TEC-PH-5816  Technology       Phones   
1  ...  FUR-CH-5379   Furniture       Chairs   
2  ...  TEC-PH-5356  Technology       Phones   
3  ...  TEC-PH-5267  Technology       Phones   
4  ...  TEC-CO-6011  Technology      Copiers   

                                Product Name      Sales Quantity Discount  \
0                           Samsung Convoy 3    $221.98        2      0.0   
1  Novimex Executive Leather Armchair, Black  $3,709.40        9      0.1   
2          Nokia Smart Phone, with Caller ID  $5,175.17        9      0.1   
3             Motorola Smart Phone, Cordless  $2,892.51        5      0.1   
4             Sharp Wireless Fax, High-Speed  $2,832.96        8      0.0   

     Profit Shipping Cost  Order Priority  
0    $62.15         40.77            High  
1  -$288.77        923.63        Critical  
2   $919.97        915.49          Medium  
3   -$96.54        910.16          Medium  
4   $311.52        903.04        Critical  

[5 rows x 24 columns]

In [97]:
# look at data types
super_df_drop.dtypes

State              object
Country            object
Category           object
Sub_Category       object
Sales              object
Profit             object
Shipping Cost     float64
Order Priority     object
dtype: object

In [83]:
# drop nonessential columns
# df = df.loc[:, df.columns.intersection(['a','b'])]
super_df_drop = super_df.drop(super_df.columns.difference(['State','Country','Category','Sub-Category', 'Sales', 'Profit', 'Shipping Cost','Order Priority']), 1)

/Users/forrestcasey/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [84]:
# fix sales values
super_df_drop['Sales'] = super_df_drop['Sales'].str.replace(',', '')
super_df_drop['Sales'] = super_df_drop['Sales'].str.replace('$', '')
super_df_drop['Sales']

/Users/forrestcasey/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until


0         221.98
1        3709.40
2        5175.17
3        2892.51
4        2832.96
          ...   
51285      65.10
51286      16.72
51287       8.56
51288      13.44
51289      61.38
Name: Sales, Length: 51290, dtype: object

In [85]:
super_df_drop.head()

State        Country    Category Sub-Category    Sales    Profit  \
0         Oklahoma  United States  Technology       Phones   221.98    $62.15   
1  New South Wales      Australia   Furniture       Chairs  3709.40  -$288.77   
2       Queensland      Australia  Technology       Phones  5175.17   $919.97   
3           Berlin        Germany  Technology       Phones  2892.51   -$96.54   
4            Dakar        Senegal  Technology      Copiers  2832.96   $311.52   

   Shipping Cost Order Priority  
0          40.77           High  
1         923.63       Critical  
2         915.49         Medium  
3         910.16         Medium  
4         903.04       Critical

In [86]:
# Determine the number of unique values in each column.
super_df_drop.nunique()

State              1102
Country             165
Category              3
Sub-Category         17
Sales             20049
Profit            16578
Shipping Cost     16452
Order Priority        4
dtype: int64

In [87]:
# Look at value counts for binning
super_df_count = super_df_drop.Category.value_counts()
super_df_count

Office Supplies    31289
Technology         10141
Furniture           9860
Name: Category, dtype: int64

In [88]:
# rename sub-category column so we can print its values
# df.rename(columns={'oldName1': 'newName1', 'oldName2': 'newName2'}, inplace=True)
super_df_drop.rename(columns={'Sub-Category': 'Sub_Category'}, inplace=True)
super_df_drop.head()

State        Country    Category Sub_Category    Sales    Profit  \
0         Oklahoma  United States  Technology       Phones   221.98    $62.15   
1  New South Wales      Australia   Furniture       Chairs  3709.40  -$288.77   
2       Queensland      Australia  Technology       Phones  5175.17   $919.97   
3           Berlin        Germany  Technology       Phones  2892.51   -$96.54   
4            Dakar        Senegal  Technology      Copiers  2832.96   $311.52   

   Shipping Cost Order Priority  
0          40.77           High  
1         923.63       Critical  
2         915.49         Medium  
3         910.16         Medium  
4         903.04       Critical

In [89]:
# make sub-category dataframe and view value counts
super_df_sub_cat = super_df_drop.Sub_Category.value_counts()
super_df_sub_cat

Binders        6146
Storage        5049
Art            4864
Paper          3492
Chairs         3434
Phones         3357
Furnishings    3154
Accessories    3075
Labels         2601
Fasteners      2601
Bookcases      2411
Supplies       2407
Envelopes      2387
Copiers        2223
Appliances     1742
Machines       1486
Tables          861
Name: Sub_Category, dtype: int64

In [90]:
# super_df_drop.groupby('Sub_Category').Sales.sum()

In [91]:
print(super_df_drop.isnull().sum())

State             0
Country           0
Category          0
Sub_Category      0
Sales             0
Profit            0
Shipping Cost     0
Order Priority    0
dtype: int64


In [92]:
super_df_drop.groupby('Category').Sales.sum()

Category
Furniture          3709.401822.085244.84341.9648.714626.152221.80...
Office Supplies    17.943701.521869.5912.627958.583409.74242.9452...
Technology         221.985175.172892.512832.962862.682616.962565....
Name: Sales, dtype: object

In [93]:
super_df_drop.head()

State        Country    Category Sub_Category    Sales    Profit  \
0         Oklahoma  United States  Technology       Phones   221.98    $62.15   
1  New South Wales      Australia   Furniture       Chairs  3709.40  -$288.77   
2       Queensland      Australia  Technology       Phones  5175.17   $919.97   
3           Berlin        Germany  Technology       Phones  2892.51   -$96.54   
4            Dakar        Senegal  Technology      Copiers  2832.96   $311.52   

   Shipping Cost Order Priority  
0          40.77           High  
1         923.63       Critical  
2         915.49         Medium  
3         910.16         Medium  
4         903.04       Critical

In [96]:
super_df_drop.dtypes

State              object
Country            object
Category           object
Sub_Category       object
Sales              object
Profit             object
Shipping Cost     float64
Order Priority     object
dtype: object